# Seminar on Machine Learning


## Literature:
https://jakevdp.github.io/PythonDataScienceHandbook/index.html

## Students file space - and my notebooks and data:
https://su.mff.cuni.cz/view/browse/home/martav/seminardm

## Expected Output for Credits:

You should submit and present a semanar work that presents your ability to use machine learning techniques to your own dataset. You select or get your own topic. The work should present at least 12 ideas (graphs, summaries, models, ...), should be encapsulated into text explanations. At least 3 ideas should be not just a 'copy' of the presentation notebooks.


## Form:
Lectures every second weeks, seminars every week.

## Oracle install:

### In the shell, run:

python -m pip install cx_Oracle --upgrade

### Download and unzip 

the Basic Package https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html

## Add the path to the command:

cx_Oracle.init_oracle_client(config_dir="/home/your_username/oracle/your_config_dir")

# Basic Libraries

In [1]:
import numpy as np
import pandas as pd
import cx_Oracle
import json
import os

#from sqlalchemy import create_engine

In [2]:
cx_Oracle.init_oracle_client(config_dir="C:\\Users\\marta\\Desktop\\Plocha\\programy\\instantclient_21_8")

import warnings
warnings.simplefilter("ignore")

# Data frame - our basic data structure

we use the DataFrame format from the pandas library

## Read a *.csv  file

demo of the *.csv read with the printout of the first record .head(1)

(covid data in Czech)

In [3]:
covid=pd.read_csv('data/covid.csv')
covid.head(1)

,datum,kumulativni_pocet_nakazenych,kumulativni_pocet_vylecenych,kumulativni_pocet_umrti,kumulativni_pocet_testu,kumulativni_pocet_ag_testu
0,2020-01-27,0,0,0,20,0


## dataFrame from scratch

you may define a dataframe directly from a dict structure

&emsp;Series

&emsp;&emsp;'column_name':[column_data]

In [4]:
my_data=pd.DataFrame({
  'age':['0-14','15-19','20-24','25-29','30-34','35-39','40-49','50-59','60-69','70-79','80 a více'],
  'male':[2299,784,854,927,1197,1329,2443,3003,4289,3322,1747],
  'female':[2159,814,950,1153,1494,1505,2627,3571,5837,5413,4094] 
})
my_data.head(1)

,age,male,female
0,0-14,2299,2159


## Data from *.json

In [5]:
movies= pd.read_json('data/movies.json')
movies.head(1)

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy\r,tt0114709,49695,45868,4,4


## Oracle connection
SQL language on demand, the basics: 

SELECT a keyword  
&emsp;     $*$ select all columns; a list of columns otherwise   
FROM a keyword   
&emsp;     MMOVIES table name
&emsp;&emsp; #        we could end here. We can add restrictions:  
    WHERE a keyword 
&emsp;     CNT>100 condition: column_name oprerator value  
    ORDER BY ...  
&emsp; 

Thanks to Michalovi Kopeckému for the data preparation and access.

In [6]:
try:
    connection =cx_Oracle.connect(user='websem', password='websem108', dsn='tirpitz.ms.mff.cuni.cz:1511/jedenact.ms.mff.cuni.cz')
    movies=pd.read_sql_query('''select * from MMOVIES 
                                order by MOVIEID
                            ''', 
                             connection,index_col='MOVIEID')
except:
    movies= pd.read_json('data/movies.json')
    print('Except')

movies.head(1)

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
MOVIEID,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy\r,tt0114709,49695,45868,4,4


In [7]:
connection =cx_Oracle.connect(user='websem', password='websem108', dsn='tirpitz.ms.mff.cuni.cz:1511/jedenact.ms.mff.cuni.cz')

In [8]:
ratings=pd.read_sql_query('''select * from FRATINGS 
                               where MOVIEID<20
                            ''', 
                             connection)

In [9]:
ratings.columns

Index(['USERID', 'MOVIEID', 'RATING', 'TS', 'DT', 'TOP5', 'K', 'DTYY', 'DTMM',
       'DTDW', 'DTHH', 'KTT', 'TRAIN', 'TEST', 'DIFF'],
      dtype='object')

In [10]:
pd.read_sql_query('''select ACTORS from IMDB''', connection)

,ACTORS
0,None
1,None
2,"Joseph Stacey, Gabi Herrett, John Grey"
3,"Chris Stuckmann, Fae Rie, Sam Liz, Trevor Lawr..."
4,Madison Marie Steinacker
...,...
40740,"Jonnie Hurn, Chloe Farnworth, Dan Burman, Lulu..."
40741,"Götz George, Uwe Ochsenknecht, Christiane Hörb..."
40742,"Merrill Gruver, Michael Edwards, Melissa Brown..."
40743,"Gabrielle Anwar, Michael Schoeffling, Cliff Ro..."


## Select a column of a dataframe

two alternatives

In [11]:
movies.AVG #movies['AVG']

MOVIEID
1         4
2         3
3         3
4         3
5         3
         ..
131254    4
131256    4
131258    0
131260    0
131262    0
Name: AVG, Length: 27278, dtype: int64

# DataFrame basic info

In [12]:
print("movies.shape is {}.".format(movies.shape))
print("Number of rows is {}, columns {}.".format(movies.shape[0], movies.shape[1]))

print("Column names are {}.".format(movies.columns))

print("... whithout type info {}.".format(movies.columns.tolist()))

print("movies.describe() output:")

movies.describe()

movies.shape is (27278, 7).
Number of rows is 27278, columns 7.
Column names are Index(['TITLE', 'GENRE', 'TT', 'CNT0', 'CNT', 'AVG', 'BAYESAVG'], dtype='object').
... whithout type info ['TITLE', 'GENRE', 'TT', 'CNT0', 'CNT', 'AVG', 'BAYESAVG'].
movies.describe() output:


,CNT0,CNT,AVG,BAYESAVG
count,27278.000000,27278.000000,27278.000000,27278.000000
mean,733.201224,682.430273,3.086810,3.268238
std,3057.219626,2838.253504,0.904714,0.779929
min,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,3.000000,3.000000
50%,16.000000,15.000000,3.000000,3.000000
75%,194.000000,182.000000,4.000000,4.000000
max,67310.000000,62744.000000,5.000000,4.000000


## Store the dataFrame in *.json format

In [13]:
with open('data/mmovies.json', 'w') as f:
    f.write(movies.to_json())

# We may access following tables

In [14]:
try:
    oracle_tables=pd.read_sql_query("SELECT * FROM all_tables WHERE OWNER='RULEML'", connection)
except:
    oracle_tables= pd.read_json('data/oracle.json')

### Here is the list

In [15]:
oracle_tables

,OWNER,TABLE_NAME,TABLESPACE_NAME,CLUSTER_NAME,IOT_NAME,STATUS,PCT_FREE,PCT_USED,INI_TRANS,MAX_TRANS,...,SKIP_CORRUPT,MONITORING,CLUSTER_OWNER,DEPENDENCIES,COMPRESSION,COMPRESS_FOR,DROPPED,READ_ONLY,SEGMENT_CREATED,RESULT_CACHE
0,RULEML,FUSERS,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
1,RULEML,TW_TRAINING,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
2,RULEML,FRATINGS,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
3,RULEML,FMOVIES,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
4,RULEML,MMOVIES,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
5,RULEML,MRATINGS,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
6,RULEML,DBP_KODY,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
7,RULEML,IMDB,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
8,RULEML,MUSERS,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT
9,RULEML,WMOVIES,RULEML,None,None,VALID,10,None,1,255,...,DISABLED,YES,None,DISABLED,DISABLED,None,NO,NO,YES,DEFAULT


### One column list is sufficient

In [16]:
oracle_tables['TABLE_NAME']

0          FUSERS
1     TW_TRAINING
2        FRATINGS
3         FMOVIES
4         MMOVIES
5        MRATINGS
6        DBP_KODY
7            IMDB
8          MUSERS
9         WMOVIES
10         WUSERS
Name: TABLE_NAME, dtype: object

We have three sets of tables 

    M-movielense, F - netflix, W - twitter
    
with three tables in each set:

    ?MOVIES, ?RATINGS, ?USERS
    
the tables are connected by the keys MOVIEID, USERID

the movies are connected by TT with

    IMDB

## Selection criteria definition (movies with many ratings)

In [17]:
popular_crit=movies.CNT>50000
popular_crit

MOVIEID
1         False
2         False
3         False
4         False
5         False
          ...  
131254    False
131256    False
131258    False
131260    False
131262    False
Name: CNT, Length: 27278, dtype: bool

## Selection according the criterion

In [18]:
popular=movies[popular_crit]
popular

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
MOVIEID,,,,,,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi\r,tt0076759,54502,50180,4,4
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller\r,tt0110912,67310,62744,4,4
318,"Shawshank Redemption, The (1994)",Crime|Drama\r,tt0111161,63366,57486,4,4
356,Forrest Gump (1994),Comedy|Drama|Romance|War\r,tt0109830,66172,60572,4,4
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller\r,tt0107290,59715,54957,4,4
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller\r,tt0102926,63299,58429,4,4


In [19]:
popular.columns

Index(['TITLE', 'GENRE', 'TT', 'CNT0', 'CNT', 'AVG', 'BAYESAVG'], dtype='object')

In [20]:
'TTT' in popular.columns

False

In [21]:
popular.TITLE  # jeden sloupec je Series

MOVIEID
260    Star Wars: Episode IV - A New Hope (1977)
296                          Pulp Fiction (1994)
318             Shawshank Redemption, The (1994)
356                          Forrest Gump (1994)
480                         Jurassic Park (1993)
593             Silence of the Lambs, The (1991)
Name: TITLE, dtype: object

### Series index and values

In [22]:
popular.TITLE.index # Series má .index  a .values

Int64Index([260, 296, 318, 356, 480, 593], dtype='int64', name='MOVIEID')

In [23]:
popular.TITLE.values

array(['Star Wars: Episode IV - A New Hope (1977)', 'Pulp Fiction (1994)',
       'Shawshank Redemption, The (1994)', 'Forrest Gump (1994)',
       'Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'],
      dtype=object)

### Dataframe index and values 

In [24]:
popular.index

Int64Index([260, 296, 318, 356, 480, 593], dtype='int64', name='MOVIEID')

In [25]:
popular.values

array([['Star Wars: Episode IV - A New Hope (1977)',
        'Action|Adventure|Sci-Fi\r', 'tt0076759', 54502, 50180, 4, 4],
       ['Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller\r',
        'tt0110912', 67310, 62744, 4, 4],
       ['Shawshank Redemption, The (1994)', 'Crime|Drama\r', 'tt0111161',
        63366, 57486, 4, 4],
       ['Forrest Gump (1994)', 'Comedy|Drama|Romance|War\r', 'tt0109830',
        66172, 60572, 4, 4],
       ['Jurassic Park (1993)', 'Action|Adventure|Sci-Fi|Thriller\r',
        'tt0107290', 59715, 54957, 4, 4],
       ['Silence of the Lambs, The (1991)', 'Crime|Horror|Thriller\r',
        'tt0102926', 63299, 58429, 4, 4]], dtype=object)

# Sequential pass through # rarely used; usually, we assign (print) as a structure

### Series

In [26]:
popular.TITLE.keys()

Int64Index([260, 296, 318, 356, 480, 593], dtype='int64', name='MOVIEID')

In [27]:
popular.TITLE.items()

### for cycle in pyton syntax

In [28]:
for (key,value) in popular.TITLE.items():
    print(value)

Star Wars: Episode IV - A New Hope (1977)
Pulp Fiction (1994)
Shawshank Redemption, The (1994)
Forrest Gump (1994)
Jurassic Park (1993)
Silence of the Lambs, The (1991)


### DataFrame: Series Series

In [29]:
popular.keys()

Index(['TITLE', 'GENRE', 'TT', 'CNT0', 'CNT', 'AVG', 'BAYESAVG'], dtype='object')

In [30]:
for (key,value) in popular.items():
    print(value)

MOVIEID
260    Star Wars: Episode IV - A New Hope (1977)
296                          Pulp Fiction (1994)
318             Shawshank Redemption, The (1994)
356                          Forrest Gump (1994)
480                         Jurassic Park (1993)
593             Silence of the Lambs, The (1991)
Name: TITLE, dtype: object
MOVIEID
260             Action|Adventure|Sci-Fi\r
296         Comedy|Crime|Drama|Thriller\r
318                         Crime|Drama\r
356            Comedy|Drama|Romance|War\r
480    Action|Adventure|Sci-Fi|Thriller\r
593               Crime|Horror|Thriller\r
Name: GENRE, dtype: object
MOVIEID
260    tt0076759
296    tt0110912
318    tt0111161
356    tt0109830
480    tt0107290
593    tt0102926
Name: TT, dtype: object
MOVIEID
260    54502
296    67310
318    63366
356    66172
480    59715
593    63299
Name: CNT0, dtype: int64
MOVIEID
260    50180
296    62744
318    57486
356    60572
480    54957
593    58429
Name: CNT, dtype: int64
MOVIEID
260    4
296    4
318

In [31]:
'TT' in popular.keys() #set operations

True

# Slicing a DataFrame

In [32]:
# slicing by explicit index
popular['TITLE']

MOVIEID
260    Star Wars: Episode IV - A New Hope (1977)
296                          Pulp Fiction (1994)
318             Shawshank Redemption, The (1994)
356                          Forrest Gump (1994)
480                         Jurassic Park (1993)
593             Silence of the Lambs, The (1991)
Name: TITLE, dtype: object

In [33]:
popular.TITLE   # pokud není v konfliktu s klíčovým slovem a je string

MOVIEID
260    Star Wars: Episode IV - A New Hope (1977)
296                          Pulp Fiction (1994)
318             Shawshank Redemption, The (1994)
356                          Forrest Gump (1994)
480                         Jurassic Park (1993)
593             Silence of the Lambs, The (1991)
Name: TITLE, dtype: object

In [34]:
popular[['TITLE','GENRE']] #list of columns

,TITLE,GENRE
MOVIEID,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi\r
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller\r
318,"Shawshank Redemption, The (1994)",Crime|Drama\r
356,Forrest Gump (1994),Comedy|Drama|Romance|War\r
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller\r
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller\r


In [35]:
popular[popular.AVG==4]

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
MOVIEID,,,,,,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi\r,tt0076759,54502,50180,4,4
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller\r,tt0110912,67310,62744,4,4
318,"Shawshank Redemption, The (1994)",Crime|Drama\r,tt0111161,63366,57486,4,4
356,Forrest Gump (1994),Comedy|Drama|Romance|War\r,tt0109830,66172,60572,4,4
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller\r,tt0107290,59715,54957,4,4
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller\r,tt0102926,63299,58429,4,4


## Compound condition OR

In [36]:
popular[(popular.index <300) | (popular.index >500)]

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
MOVIEID,,,,,,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi\r,tt0076759,54502,50180,4,4
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller\r,tt0110912,67310,62744,4,4
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller\r,tt0102926,63299,58429,4,4


## loc

reference by the index value


In [37]:
popular.loc[260]

TITLE       Star Wars: Episode IV - A New Hope (1977)
GENRE                       Action|Adventure|Sci-Fi\r
TT                                          tt0076759
CNT0                                            54502
CNT                                             50180
AVG                                                 4
BAYESAVG                                            4
Name: 260, dtype: object

## interval reference

In [38]:
popular.loc[420:800]

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
MOVIEID,,,,,,,
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller\r,tt0107290,59715,54957,4,4
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller\r,tt0102926,63299,58429,4,4


## iloc  reference by the position

In [39]:
popular.iloc[0:2]

,TITLE,GENRE,TT,CNT0,CNT,AVG,BAYESAVG
MOVIEID,,,,,,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi\r,tt0076759,54502,50180,4,4
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller\r,tt0110912,67310,62744,4,4


In [40]:
popular.iloc[:3,:4]

,TITLE,GENRE,TT,CNT0
MOVIEID,,,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi\r,tt0076759,54502
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller\r,tt0110912,67310
318,"Shawshank Redemption, The (1994)",Crime|Drama\r,tt0111161,63366


## selection of both rows and columns

In [41]:
(popular[popular.CNT>=60000])[['TITLE','CNT']]

,TITLE,CNT
MOVIEID,,
296,Pulp Fiction (1994),62744
356,Forrest Gump (1994),60572


# 2D array

In [42]:
popular.values

array([['Star Wars: Episode IV - A New Hope (1977)',
        'Action|Adventure|Sci-Fi\r', 'tt0076759', 54502, 50180, 4, 4],
       ['Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller\r',
        'tt0110912', 67310, 62744, 4, 4],
       ['Shawshank Redemption, The (1994)', 'Crime|Drama\r', 'tt0111161',
        63366, 57486, 4, 4],
       ['Forrest Gump (1994)', 'Comedy|Drama|Romance|War\r', 'tt0109830',
        66172, 60572, 4, 4],
       ['Jurassic Park (1993)', 'Action|Adventure|Sci-Fi|Thriller\r',
        'tt0107290', 59715, 54957, 4, 4],
       ['Silence of the Lambs, The (1991)', 'Crime|Horror|Thriller\r',
        'tt0102926', 63299, 58429, 4, 4]], dtype=object)

In [43]:
popular.T

MOVIEID,260,296,318,356,480,593
TITLE,Star Wars: Episode IV - A New Hope (1977),Pulp Fiction (1994),"Shawshank Redemption, The (1994)",Forrest Gump (1994),Jurassic Park (1993),"Silence of the Lambs, The (1991)"
GENRE,Action|Adventure|Sci-Fi\r,Comedy|Crime|Drama|Thriller\r,Crime|Drama\r,Comedy|Drama|Romance|War\r,Action|Adventure|Sci-Fi|Thriller\r,Crime|Horror|Thriller\r
TT,tt0076759,tt0110912,tt0111161,tt0109830,tt0107290,tt0102926
CNT0,54502,67310,63366,66172,59715,63299
CNT,50180,62744,57486,60572,54957,58429
AVG,4,4,4,4,4,4
BAYESAVG,4,4,4,4,4,4


In [44]:
popular.values[0]

array(['Star Wars: Episode IV - A New Hope (1977)',
       'Action|Adventure|Sci-Fi\r', 'tt0076759', 54502, 50180, 4, 4],
      dtype=object)

# Data for you seminar tasks:

In [45]:
try:
    connection =cx_Oracle.connect(user='websem', password='websem108', 
                                  dsn='tirpitz.ms.mff.cuni.cz:1511/jedenact.ms.mff.cuni.cz')
    ratings=pd.read_sql_query('select * from MRATINGS WHERE MOVIEID<20',connection,index_col=['MOVIEID','USERID'])
except:
    ratings= pd.read_json('data/ratings.json')

ratings.head()

RATING          TS                  DT  TOP5    K  DTYY  DTMM  \
MOVIEID USERID                                                                  
1       3          4.0   944919407 1999-12-11 13:36:47  None   81  1999    12   
        6          5.0   858275452 1997-03-13 17:50:52  None   18  1997     3   
        8          4.0   833981871 1996-06-05 13:37:51  None   45  1996     6   
        10         4.0   943497887 1999-11-25 02:44:47  None    5  1999    11   
        11         4.5  1230858821 2009-01-02 01:13:41  None  138  2009     1   

                DTDW  DTHH  KTT TRAIN TEST  DIFF  
MOVIEID USERID                                    
1       3          7    13   81     A    N   0.0  
        6          5    17   18     A    N   1.0  
        8          4    13   45     A    N   0.0  
        10         5     2    5     N    A   NaN  
        11         6     1  138     A    N   0.5

In [46]:
ratings.reset_index(inplace=True)

In [47]:
ratings

,MOVIEID,USERID,RATING,TS,DT,TOP5,K,DTYY,DTMM,DTDW,DTHH,KTT,TRAIN,TEST,DIFF
0,1,3,4.0,944919407,1999-12-11 13:36:47,None,81,1999,12,7,13,81,A,N,0.0
1,1,6,5.0,858275452,1997-03-13 17:50:52,None,18,1997,3,5,17,18,A,N,1.0
2,1,8,4.0,833981871,1996-06-05 13:37:51,None,45,1996,6,4,13,45,A,N,0.0
3,1,10,4.0,943497887,1999-11-25 02:44:47,None,5,1999,11,5,2,5,N,A,NaN
4,1,11,4.5,1230858821,2009-01-02 01:13:41,None,138,2009,1,6,1,138,A,N,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267427,19,138406,4.0,1155426512,2006-08-12 23:48:32,None,347,2006,8,7,23,347,A,N,1.0
267428,19,138474,5.0,966406535,2000-08-16 06:15:35,None,64,2000,8,4,6,64,A,N,2.0
267429,19,138483,1.0,833831393,1996-06-03 19:49:53,None,240,1996,6,2,19,240,A,N,-2.0
267430,19,138484,2.0,836154340,1996-06-30 17:05:40,None,117,1996,6,1,17,117,A,N,-1.0


In [48]:
with open('data/ratings.json', 'w') as f:
    f.write(ratings.to_json())

In [49]:
rat=pd.read_json('data/ratings.json')
rat.head()

,MOVIEID,USERID,RATING,TS,DT,TOP5,K,DTYY,DTMM,DTDW,DTHH,KTT,TRAIN,TEST,DIFF
0,1,3,4.0,944919407,944919407000,NaN,81,1999,12,7,13,81,A,N,0.0
1,1,6,5.0,858275452,858275452000,NaN,18,1997,3,5,17,18,A,N,1.0
2,1,8,4.0,833981871,833981871000,NaN,45,1996,6,4,13,45,A,N,0.0
3,1,10,4.0,943497887,943497887000,NaN,5,1999,11,5,2,5,N,A,NaN
4,1,11,4.5,1230858821,1230858821000,NaN,138,2009,1,6,1,138,A,N,0.5


# Tasks for the first seminar
1. Read the table RATINGS  (data/ratings.json or from the Oracle database).
3. Write down the list of columns. Answer with a sentence, not just the print.
4. How many rows has the dataframe?
6. How many records with MOVEID<20  and RATING==5 ?
7. What movies are rated by the user USERID ==11 ?
8. Put your name into a markdown comment and submit the file into https://su.mff.cuni.cz/view/browse/home/martav/seminardm/students/seminar1 .

